## Big Spender Porfolio Calculator

Allocate a minimal budget that buys at least one share of each ticker and produces as-even-as-possible dollar weights across tickers.

In [1]:
from typing import List, Optional, Tuple, Dict


def allocate_min_budget_even_weights(
    prices: List[float],
    tickers: Optional[List[str]] = None,
    *,
    max_iters: int = 10000,
    tiny: float = 1e-12,
) -> Tuple[Dict[str, int], float, float]:
    """Compute a minimal budget allocation that buys at least one share of
    each ticker and produces as-even-as-possible dollar weights across tickers.

    Algorithm (greedy, incremental):
    - Start with one share of each stock (minimal requirement). Spent = sum(prices).
    - Compute imbalance metric = sum(|weight_i - mean_weight|).
    - Repeatedly consider adding one share to each stock and pick the single-share
      purchase that yields the largest reduction in imbalance per dollar spent.
    - Stop when no single-share purchase reduces the imbalance.

    This yields a minimal extra spend (greedy heuristic) that improves balance
    until no single increment helps.
    Returns (allocations, spent, leftover) where leftover is 0.0 because we count
    the budget as the exact amount spent to reach the allocation.
    """
    if not prices:
        return ({}, 0.0, 0.0)

    n = len(prices)
    if tickers is None:
        keys = [str(i) for i in range(n)]
    else:
        if len(tickers) != n:
            raise ValueError("tickers length must match prices length")
        keys = list(tickers)

    qtys = [1 for _ in prices]
    spent = sum(float(p) for p in prices)

    def imbalance(qtys_local: List[int], spent_local: float) -> float:
        weights = [q * p for q, p in zip(qtys_local, prices)]
        mean = spent_local / n
        return sum(abs(w - mean) for w in weights)

    cur_metric = imbalance(qtys, spent)

    it = 0
    while it < max_iters:
        it += 1
        best = None  # (score, index, new_metric)
        for i in range(n):
            price = float(prices[i])
            new_spent = spent + price
            # weights after adding one share to i
            new_weights = [q * p for q, p in zip(qtys, prices)]
            new_weights[i] += price
            new_mean = new_spent / n
            new_metric = sum(abs(w - new_mean) for w in new_weights)
            improvement = cur_metric - new_metric
            if improvement > tiny:
                score = improvement / price
                if best is None or score > best[0]:
                    best = (score, i, new_metric)
        if best is None:
            break
        _, idx, new_metric = best
        qtys[idx] += 1
        spent += float(prices[idx])
        cur_metric = new_metric

    allocations = {k: q for k, q in zip(keys, qtys)}
    # We treat the spent amount as the effective budget (no leftover)
    return allocations, round(spent, 10), 0.0


# Example usage
tickers = ["ACN", "CMG", "DHR", "ODFL", "UPS", "GIS"]
prices = [241.24, 39.76, 192.94, 141.79, 84.18, 49.18]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ACN': 1, 'CMG': 5, 'DHR': 1, 'ODFL': 1, 'UPS': 2, 'GIS': 4}
Spent: USD 1139.85


In [2]:
tickers = ["ADBE", "FISV", "ACN", "CMG", "DHR", "ODFL"]
prices = [363.10, 129.92, 239.53, 39.89, 190.64, 139.17]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ADBE': 1, 'FISV': 2, 'ACN': 1, 'CMG': 7, 'DHR': 1, 'ODFL': 2}
Spent: USD 1610.68


In [10]:
tickers = ["ACN", "ADBE", "CMCSA", "FISV", "OKE", "SLB"]
prices = [240.94, 337.51, 29.46, 122.63, 69.09, 31.72]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ACN': 1, 'ADBE': 1, 'CMCSA': 9, 'FISV': 2, 'OKE': 4, 'SLB': 9}
prices: [240.94, 337.51, 29.46, 122.63, 69.09, 31.72]
Spent: USD 1650.69


In [11]:
tickers = ["ADBE", "FISV", "CMCSA", "ACN", "OKE", "ODFL"]
prices = [333.26, 121.65, 29.54, 238.39, 68.09, 138.04]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ADBE': 1, 'FISV': 2, 'CMCSA': 9, 'ACN': 1, 'OKE': 4, 'ODFL': 2}
prices: [333.26, 121.65, 29.54, 238.39, 68.09, 138.04]
Spent: USD 1629.25


In [13]:
# Week from November 3 to November 7, 2025
tickers = ["ADBE", "CHTR", "FISV", "CMG", "STZ", "CMCSA"]
prices = [340.31, 233.84, 66.69, 31.69, 131.38, 27.84]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ADBE': 1, 'CHTR': 1, 'FISV': 4, 'CMG': 8, 'STZ': 2, 'CMCSA': 9}
prices: [340.31, 233.84, 66.69, 31.69, 131.38, 27.84]
Spent: USD 1607.75


In [18]:
# Week from November 10 to November 14, 2025
tickers = ["ADBE", "ZTS", "CHTR", "FISV", "CRM", "CMG"]
prices = [326.95, 120.24, 219.86, 63.70, 239.88, 30.59]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ADBE': 1, 'ZTS': 2, 'CHTR': 1, 'FISV': 4, 'CRM': 1, 'CMG': 8}
prices: [326.95, 120.24, 219.86, 63.7, 239.88, 30.59]
Spent: USD 1526.69


In [ ]:
# Week from November 17 to November 21, 2025
tickers = ["ADBE", "ZTS", "CPRT", "CHTR", "CMCSA", "FISV"]
prices = [331.11, 120.82, 41.08, 203.10, 27.51, 63.8]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ADBE': 1, 'ZTS': 2, 'CPRT': 6, 'CHTR': 1, 'CMCSA': 9, 'FISV': 4}
prices: [331.11, 120.82, 41.08, 203.1, 27.51, 63.8]
Spent: USD 1525.12


In [20]:
# Week from November 24 to November 28, 2025
tickers = ["ZTS", "ADBE", "CHRT", "CRM", "MSI", "PYPL"]
prices = [122.06, 324.19, 203, 227.11, 368.99, 60.57]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'ZTS': 2, 'ADBE': 1, 'CHRT': 1, 'CRM': 1, 'MSI': 1, 'PYPL': 4}
prices: [122.06, 324.19, 203, 227.11, 368.99, 60.57]
Spent: USD 1609.69


In [22]:
# Week from DECEMBER 1 to DECEMBER 5, 2025
tickers = ["CRM", "MSI", "GDDY", "ODFL", "TTD", "GIS"]
prices = [230.54, 368.68, 127.86, 135.29, 39.56, 47.35]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'CRM': 1, 'MSI': 1, 'GDDY': 2, 'ODFL': 2, 'TTD': 7, 'GIS': 6}
prices: [230.54, 368.68, 127.86, 135.29, 39.56, 47.35]
Spent: USD 1686.54


In [23]:
# Week from DECEMBER 8 to DECEMBER 12, 2025
tickers = ["CHTR", "GDDY", "CMCSA", "ZTS", "CPRT", "PAYPL"]
prices = [205.1, 128.46, 27.31, 118.65, 38.72, 62.28]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print("Greedy minimal allocation (one+ even weights):")
print("Allocations:", alloc_min)
print("prices:", prices)
print("Spent: USD", spent_min)

Greedy minimal allocation (one+ even weights):
Allocations: {'CHTR': 1, 'GDDY': 1, 'CMCSA': 5, 'ZTS': 1, 'CPRT': 3, 'PAYPL': 2}
prices: [205.1, 128.46, 27.31, 118.65, 38.72, 62.28]
Spent: USD 829.48
